In [35]:
import tensorflow as tf
import numpy
import pandas as pd
from sklearn import preprocessing
numpy.random.seed(10)
import matplotlib.pyplot as plt
import os
import fileinput

In [36]:
TFLITE_MODEL = "spectrum_color_detection.tflite"

In [37]:
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 121]
type: <class 'numpy.float32'>

== Output details ==
shape: [1 4]
type: <class 'numpy.float32'>


In [38]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (8, 121))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (5, 4))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: dense_2_input
shape: [  8 121]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: [8 4]
type: <class 'numpy.float32'>


In [39]:
test_df = pd.read_excel("data/test_tflite.xls")

In [40]:
test_df.shape

(8, 121)

In [41]:
def PreprocessData(raw_df):    
    ndarray = raw_df.values  
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures=minmax_scale.fit_transform(raw_df)    
    return scaledFeatures

In [42]:
test_df=PreprocessData(test_df)

In [43]:
test_df = numpy.array(test_df, dtype=numpy.float32)

In [44]:
tflite_interpreter.set_tensor(input_details[0]['index'], test_df)

tflite_interpreter.invoke()

tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

Prediction results shape: (8, 4)


In [46]:
tflite_pred_dataframe = pd.DataFrame(tflite_model_predictions)
tflite_pred_dataframe.columns = ["red","green","blue","white"]

print("TFLite prediction results for the first elements")
tflite_pred_dataframe.head()

TFLite prediction results for the first elements


,red,green,blue,white
0,7.438326e-06,9.999608e-01,2.047848e-05,1.131420e-05
1,2.536755e-06,9.999826e-01,9.489088e-06,5.337633e-06
2,9.999998e-01,2.949473e-08,7.283118e-08,7.768128e-08
3,9.999994e-01,9.325165e-08,2.519435e-07,2.408985e-07
4,1.674271e-09,9.112871e-08,9.999999e-01,1.243983e-08


In [47]:
tflite_pred_dataframe

,red,green,blue,white
0,7.438326e-06,9.999608e-01,2.047848e-05,1.131420e-05
1,2.536755e-06,9.999826e-01,9.489088e-06,5.337633e-06
2,9.999998e-01,2.949473e-08,7.283118e-08,7.768128e-08
3,9.999994e-01,9.325165e-08,2.519435e-07,2.408985e-07
4,1.674271e-09,9.112871e-08,9.999999e-01,1.243983e-08
5,7.765810e-08,8.258030e-07,9.999988e-01,2.588014e-07
6,1.368679e-05,3.816193e-06,7.154209e-06,9.999753e-01
7,5.191080e-05,9.637854e-06,2.117801e-05,9.999173e-01
